In [ ]:

import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict

filename = 'osm/sample.osm'


In [ ]:
def count_tags(filename):
    osm_file = open(filename, "r")
    print ('opened_file')
    tagsdict = defaultdict (lambda: 0)
    #for event, elem in ET.iterparse(osm_file, events=('start', )):
    #    tagsdict[elem.tag]=tagsdict[elem.tag]+1
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            tagsdict[elem.tag]=tagsdict[elem.tag]+1
            root.clear()    
 
    return dict(tagsdict)

tags = count_tags(filename)
pprint.pprint(tags)    


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Get a list of all the tags and their count
"""

def count_tags(filename):
    osm_file = open(filename, "r")
    tagsdict = defaultdict (lambda: 0)
    for event, elem in ET.iterparse(osm_file, events=('start', )):
        tagsdict[elem.tag]=tagsdict[elem.tag]+1
    return dict(tagsdict)

tags = count_tags(filename)
pprint.pprint(tags)    


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
check that the tag keys are valid chars
"""
import re
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}


osm_file = open(filename, "r")
context = iter(ET.iterparse(osm_file, events=('start', 'end')))
_, root = next(context)
for event, element in context:
    if event == 'end':
        if element.tag == "tag":
            key = element.attrib['k']
            whichCase = "other"
            if (lower.match(key)): whichCase = "lower"
            elif (lower_colon.match(key)): whichCase = "lower_colon"
            elif (problemchars.search(key)): whichCase = "problemchars"
            #print key, whichCase
            keys [whichCase] = keys [whichCase] + 1
    root.clear()
    
pprint.pprint(keys)

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
get the unique users that have modified the map
"""

def get_attrib(element, key):
    if ( key in element.attrib ):
        return element.attrib[key]
    return None

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user  = get_attrib(element, 'user')
        if user != None: users.add(user)
    return users

users = process_map(filename)
pprint.pprint(users)


In [ ]:
"""
Audit Street Types
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
first_word_re = re.compile(r'^\w+', re.IGNORECASE)
old_expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Terrace", "Loop", "Highway", "Course", "Circle", "Way", "Crescent", "Walk"]
expected_first = ["Avenue"]
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Rd." : "Road",
            "Rd" : "Road"
            }
extra_suffix = ["North", "East", "West", "South"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            #print 'Found', street_type
            if street_type_first(street_name): return
            else: street_types[street_type].add(street_name)
            
def street_type_first(street_name):
    #print 'Check first', street_name
    m = first_word_re.search(street_name)
    if m:
        if m.group() in expected_first:
            #print 'Found Street Type first for', street_name
            return True
    return False
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street_types_in_file(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    #print tag.attrib['k'], tag.attrib['v']
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        
        #print "Checking ", name
        if street_type in expected: pass
        elif street_type in mapping: 
            print "Fixing name ", name
            name = street_type_re.sub(mapping[street_type], name)
            #print "Fixed name ", name
        else:
            processed_suffix = process_suffix(name, mapping)
            if processed_suffix != None: return processed_suffix
        
        return name     

def process_suffix (name, mapping):
    split_name = name.split(' ')
    if len(split_name) > 2 and split_name[-1] in extra_suffix:
        print "Has a valid suffix ", name
        return update_name(name.rsplit(' ', 1)[0], mapping) + ' ' + split_name[-1] 
    return None

unexpected_st_types = audit_street_types_in_file(filename)
pprint.pprint(dict(unexpected_st_types))

for st_type, ways in unexpected_st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name


In [ ]:
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
replace_tags = {'addr':'address', 'type': 'type_as_specified'}
has_value_and_children = ['hgv', 'name', 'building', 'railway', 'lanes', 'maxspeed', 'source']

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [ ]:
def process_attributes(element):
    attributes = {}
    created = {}
    #address = {}
    pos = [None,None]

    for attribute in element.attrib:
        value = element.attrib[attribute]
        #print attribute

        if attribute in CREATED:
            created[attribute] = value

        elif attribute == 'lat': 
            try:
                pos [0] = float(value)
            except ValueError:
                print "Not a float"                
        elif attribute == 'lon':
            try:
                pos [1] = float(value)
            except ValueError:
                print "Not a float"

        else: attributes[attribute] = value    

    attributes['created'] = created
    if None not in pos: attributes['pos'] = pos
        
    return attributes    

def process_refs(element):
    node_refs = []
    for ref in element.iter("nd"):
        node_refs.append(ref.attrib['ref'])
    if len(node_refs) > 0 : return {"node_refs":node_refs}
    return {}

def process_tags(element):
    tags = {}
    for tag in element.iter("tag"):
        process_tag(tag.attrib['k'],tag.attrib['v'], tags)
    return tags

In [ ]:
def process_tag (name, value, tags):
    #tags[name]=value
    #if (name.startswith('hgv')): print name, value
    try:
        if (name!= None and len(name)>0 and name.strip()!=':' and 
            name.strip()!='' and not problemchars.search(name)):
            
            levels = name.split(':')
            
            #extract the top level key and convert it as needed
            top = levels[0]
            top = replace_tags.get(top,top)
            
            #assign value if it's a top level attribute
            if len(levels) == 1: 
                if top not in tags: tags[top] = value
                else: 
                    print top, value, ' other value already found as ', tags[top]
                
            #add to address dict if there are two levels
            elif top=='address' and len(levels) == 2:
                if 'address' not in tags: tags['address']={}                    
                if not isinstance(tags['address'],dict): 
                    print 'address:', name, value, ' overwriting simple value already found as ', tags['address']
                    tags['address']={}
                process_tag (levels[1],value,tags['address'])
                
            #add to a dict if there are multiple levels
            elif top!='address' and len(levels) > 1:
                if top in has_value_and_children: top = top + "_data"
                elif top in tags and not isinstance(tags[top],dict): 
                    #a root value was already found for this tag.
                    print name, value, ' unexpected - simple value already found as ', top, tags[top]
                    return
                if top not in tags: tags[top]={}                    
                process_tag (name.split(':', 1)[1], value, tags[top])
                
    except:
        print 'Exception', name, value
        pprint.pprint(tags)
        return

In [ ]:
street_tags = ['street']
def fix_street(tags):
    if 'address' in tags:
        addr = tags['address']
        for key in addr:
            if key in street_tags:
                #print 'checking', addr[key]
                addr[key] = update_name(addr[key], mapping)

In [ ]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['type']=element.tag        
        refs = process_refs(element)
        node.update(refs)
        attrs = process_attributes(element)    
        node.update(attrs)
        tags = process_tags(element)
        fix_street(tags)
        node.update(tags)
              
        #pprint.pprint( node )
        return node
    else:
        return None

In [ ]:
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

x = process_map(filename, False)

In [ ]:
import json

def extract_tags(node,parent,tags):
    for key in node:
        #print key, node[key]
        if isinstance (node[key],dict): 
            #print "found dict for ", key
            extract_tags(node[key] , parent + key + ".", tags)
        else:
            if key not in tags: tags[key] = set()
            tags[key].add( parent+key )

def check_for_unique_tags(jsonFile):
    f = open(jsonFile)
    tags = defaultdict(set)
    for line in iter(f):
        #print line
        node = json.loads(line)
        extract_tags(node, '', tags)
    f.close()
    #pprint.pprint (dict(tags))
    multis = {k: v for k, v in (dict(tags)).items() if len(tags[k])>1}
    pprint.pprint (multis)
    
check_for_unique_tags(filename + '.json')


In [ ]:
def isfloat(value):
  try:
    float(value)
    return True
  except:
    return False

def isint(value):
  try:
    int(value)
    return True
  except:
    return False

def extract_data_types(node,parent,tags):
    for key in node:
        location = parent + key
        #print key, node[key]
        if isinstance (node[key],dict): 
            #print "found dict for ", key
            extract_data_types(node[key] , location + ".", tags)
        else:
            if location not in tags: tags[location] = set()
            tags[location].add( get_data_type(node[key]) )

def get_data_type(value):
    if value == "" or value == "NULL":
        return type(None)
    elif type(value) is list:
        return type([])
    elif isint(value):
        return type(int())
    elif isfloat(value):
        return type(float())
    else:
        return type(str())

def audit_data_types_in_file(jsonFile):
    f = open(jsonFile)
    tags = defaultdict(set)
    for line in iter(f):
        #print line
        node = json.loads(line)
        extract_data_types(node, '', tags)
    f.close()
    return dict(tags)
    
data_types = audit_data_types_in_file(filename + '.json')

In [ ]:

pprint.pprint(data_types)

In [ ]:
def pick_data_type(data_types):
    pick_type = {}
    for key in data_types:
        if  type([]) in data_types[key]:
            pick_type [key] = type([])
        elif  type(str()) in data_types[key]:
            pick_type [key] = type(str())
        elif  type(float()) in data_types[key]:
            pick_type [key] = type(float())
        elif  type(int()) in data_types[key]:
            pick_type [key] = type(int())
        else:
            pick_type [key] =  type(str())
    return pick_type

data_types = audit_data_types_in_file(filename + '.json')
save_as_data_type = pick_data_type(data_types)
#pprint.pprint (save_as_data_type)

ints = [ key for (key, value) in save_as_data_type.items() if value == type(int()) and not 'zip' in key]
print ints
#not_ints = [ key for key in ints if key == type(int()) ]

In [ ]:
def get_db():
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    # 'examples' here is the database name. It will be created if it does not exist.
    db = client.osm
    return db

db = get_db()
db.sample.find_one()

#db.test.update({field:"string"}, {$set:{field:23}})

In [ ]:
db.sample.find_one({"address.postcode" : { "$exists" : True } })


In [ ]:
def make_zip_pipeline():
    # complete the aggregation pipeline
    
    #match brasilia time zone and number of tweets > 100
    #project to _id, followers, screen_name, tweets
    #sort by followers descenging
    
    pipeline = [ 
        { "$match": { "address.postcode" : { "$exists" : True } } },
        { "$group" : { "_id": "$address.postcode", "count" : {"$sum" : 1 }}},
        { "$sort": {"_id":1 }} ]
        #{ "$sort": {"address.postcode":-1 }},
        #{ "$limit" : 1 } ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.sample.aggregate(pipeline)]


db = get_db()
pipeline = make_zip_pipeline()
result = aggregate(db, pipeline)
#import pprint
#pprint.pprint(result)
print result

In [ ]:
res = db.sample.find({"address.postcode" : '08816'})
for rec in res:
    pprint.pprint (rec)

In [ ]:
pprint.pprint(db.sample.find_one({"address.postcode" : '08854'}))

In [ ]:
wrong_length = [ rec['_id'] for rec in result if len(rec['_id']) != 5]

##for rec in result:
##    zip = rec['_id']
##    wrong_length.append(zip)

print wrong_length

In [ ]:
pprint.pprint(db.sample.find_one({"address.postcode" : '115422'}))

In [ ]:
res = db.sample.find({"pos" : None})
print res.count()
res = db.sample.find()
print res.count()

In [ ]:
def make_distinct_with_count_pipeline( by_field ):    
    pipeline = [ 
        { "$match": { by_field : { "$exists" : True } } },
        { "$group" : { "_id": "$" + by_field , "count" : {"$sum" : 1 }}},
        { "$sort": {"count":-1 }} ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.sample.aggregate(pipeline)]

def distinct_with_count(field_name):
    db = get_db()
    pipeline = make_distinct_with_count_pipeline(field_name)
    result = aggregate(db, pipeline)
    return result

#import pprint
#pprint.pprint(result)
result = distinct_with_count('address.state')
print result

In [ ]:
def make_countdistinct_pipeline( by_field ):    
    pipeline = [ 
        { "$match": { by_field : { "$exists" : True } } },
        { "$group" : { "_id": "$" + by_field }},
        { "$group": { "_id": by_field, "count" : {"$sum" : 1 } } } ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.sample.aggregate(pipeline)]

def count_distinct(field_name):
    db = get_db()
    pipeline = make_countdistinct_pipeline(field_name)
    result = aggregate(db, pipeline)
    return result[0]

#import pprint
#pprint.pprint(result)
result = count_distinct('address.state')
print result

In [ ]:
res = db.sample.find({"address.state" : {"$exists": True , "$ne": 'NY'}})
#print res
for rec in res:
    pprint.pprint (rec)

In [ ]:
db = get_db()
pipeline = make_countdistinct_pipeline('address.city')
result = aggregate(db, pipeline)
#import pprint
pprint.pprint(result)

In [ ]:
res = db.sample.find({"$and":[{"tiger.name_base" : {"$exists": True }}, 
                              {"address.street": {"$exists": True}} ] })
print res.count()
#for rec in res:
#    pprint.pprint (rec)

res = db.sample.find_one({"tiger.name_base": {"$exists": True}})
pprint.pprint(res)

matchy = db.sample.find({"address.street": {"$regex" : "82"} })
print matchy.count()
for rec in matchy: pprint.pprint (rec)

In [ ]:
def cross_check_ways ():
    no_match = {}
    maybe_match = {}
    res = db.sample.find({"tiger.name_base": {"$exists": True}})
    print "tiger #", res.count()
    for rec in res:
        name = rec["tiger"]["name_base"]
        matchy = db.sample.find({"address.street": {"$regex" : name} })
        if matchy.count()>0: 
            if name not in maybe_match: 
                maybe_match[name] = [a for a in matchy]
            maybe_match[name].insert(0,rec)
        else: no_match[name] = rec
            
    print "maybe match #", len(maybe_match)
    print "no match #", len(no_match)
    return maybe_match, no_match

(maybe_match, no_match) = cross_check_ways()
#pprint.pprint(maybe_match[maybe_match.keys()[0]])


In [ ]:
for street in maybe_match.keys() [:2]:
    print '*****  ' , street, '  *****'
    for rec in maybe_match[street]:
        try: 
            x = rec['address']['street'] + ' ' + '(address)'
            print x
        except: pass
        try:
            x = rec['tiger']['name_direction_prefix']+ ' ' + rec['tiger']['name_base']+ ' ' +  rec['tiger']['name_type']+ ' ' +  '(tiger)'
            print x
        except: pass
        try:
            x = rec['tiger']['name_base']+ ' ' +  rec['tiger']['name_type']+ ' ' +  '(tiger)'
            print x
        except: pass
    
    

In [ ]:
size = db.sample.count()
print size

In [ ]:
types = count_distinct("type")
pprint.pprint(types)

In [ ]:
def print_count_distincts(fields):
    for field_name in fields:
        #pprint.pprint(get_top(field_name))
        count_dist = count_distinct(field_name)
        print "Distinct ", field_name, count_dist['_id'], ' - count', count_dist['count']

get_count_distincts_fields = ['created.user', 'address.postcode', 'address.city', 'amenity','source']
print_count_distincts(get_count_distincts_fields)

In [ ]:
def make_get_top_pipeline( by_field ):    
    pipeline = [ 
        { "$match": { by_field : { "$exists" : True } } },
        { "$group" : { "_id": "$" + by_field , "count" : {"$sum" : 1 }}},
        { "$sort": { "count" :-1 }},
        { "$limit" : 1 } ]
    return pipeline

def get_top(field_name):
    db = get_db()
    pipeline = make_get_top_pipeline(field_name)
    result = aggregate(db, pipeline)
    return result[0]

def print_tops(fields):
    for field_name in fields:
        #pprint.pprint(get_top(field_name))
        top = get_top(field_name)
        print "Top", field_name, top['_id'], ' - count', top['count']

#import pprint
#pprint.pprint(result)
get_top_fields = ['created.user', 'address.postcode', 'address.city', 'amenity','source']
print_tops(get_top_fields)


#([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$sort":{"count":­1}}, {"$limit":1}])

In [ ]:
distinct_with_count("cuisine")

In [ ]:
distinct_with_count("source")

In [ ]:
get_distincts = ["cuisine", "source", "leisure", "office", "service", "shelter", "shop", "sport", "gnis.feature_type"]

for field_name in get_distincts:
    print "Distinct", field_name
    pprint.pprint (distinct_with_count(field_name))
    print " "
